In [172]:
# Load model

from cobra.io import read_sbml_model, write_sbml_model
#path = 'Chlamy_model_iCre1355-master/iCre1355_SBML_Matlab_files/iCre1355_auto.xml'
path_sbml3fbc = 'iCre1355_auto_fbc.xml'
#model = read_sbml_model(path)
#write_sbml_model(model, path_sbml3fbc)  # stores as SBML3 + fbc
model = read_sbml_model(path_sbml3fbc)  # magic, no warnings

# NOTE: I had to re-write the model because it gave me A LOT of warnings and made everything go super slow.

No objective coefficients in model. Unclear what should be optimized


In [3]:
model

Name,iCre1355
Memory address,0x07f8651cd1820
Number of metabolites,1845
Number of reactions,2394
Number of groups,117
Objective expression,0
Compartments,"Cytosol, Chloroplast, Mitochondria, Glyoxysome, Flagellum, Extra-organism, Nucleus, Golgi Apparatus, Eyespot, Thylakoid Lumen, Inner Mitochondrial membrane space"


In [9]:
# Assess model with memote

!memote report snapshot --filename "iCre1355_auto_fbc.html" "/Users/ericbf10/Library/CloudStorage/GoogleDrive-ericbaufa10@gmail.com/My Drive/PERSONAL/DTU/DTU - Bioinformatics and Systems Biology/2022_Autumn_Semester/27410_Computer-Aided_Cell_Factory_Design/Project/iCre1355_auto_fbc.xml"

error: No objective coefficients in model. Unclear what should be optimized
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
============================= test session starts ==============================
platform darwin -- Python 3.8.2, pytest-4.6.11, py-1.11.0, pluggy-0.13.1
rootdir: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/memote/suite/tests
plugins: anyio-3.6.1
collected 164 items / 1 skipped / 163 selected                                 

../../../../../../../../../../../../Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/memote/suite/tests/test_annotation.py F [  0%]
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF..         [ 39%]
../../../../../../../../../../../../Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/memote/suite/tests/test_basic.py . [ 40%]
.....F........F.....FF                             

In [10]:
!memote report snapshot --filename "iRC1080.html" "/Users/ericbf10/Library/CloudStorage/GoogleDrive-ericbaufa10@gmail.com/My Drive/PERSONAL/DTU/DTU - Bioinformatics and Systems Biology/2022_Autumn_Semester/27410_Computer-Aided_Cell_Factory_Design/Project/iRC1080.xml"

error: No objective coefficients in model. Unclear what should be optimized
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
============================= test session starts ==============================
platform darwin -- Python 3.8.2, pytest-4.6.11, py-1.11.0, pluggy-0.13.1
rootdir: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/memote/suite/tests
plugins: anyio-3.6.1
collected 164 items / 1 skipped / 163 selected                                 

../../../../../../../../../../../../Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/memote/suite/tests/test_annotation.py . [  0%]
..FFFFFFFF..FFFFF.FFFFFFFFFFFFFFF..FF..F...F....FFF.FFFFFFFFFF..         [ 39%]
../../../../../../../../../../../../Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/memote/suite/tests/test_basic.py . [ 40%]
.....F........F...F.FF                             

In [35]:
# Explore the model

print("The model has %d reactions and %d metabolites" % (len(model.reactions), len(model.metabolites)))

The model has 2394 reactions and 1845 metabolites


In [36]:
# Create the stoichiometric matrix

from cobra.util import create_stoichiometric_matrix
S = create_stoichiometric_matrix(model)
print(S)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [37]:
# Check non-zero elements of S

import numpy as np
non_zero_elem = np.count_nonzero(S)
zero_elem = np.count_nonzero(S==0)
print("The percentage of non-zero elements in S is:", str(non_zero_elem/(zero_elem + non_zero_elem)*100) + "%")

The percentage of non-zero elements in S is: 0.2494492781185122%


In [55]:
for reaction in model.reactions.query('Biomass', 'name'):
    print(reaction.id)

Biomass_Chlamy_auto
Biomass_Chlamy_mixo
Biomass_Chlamy_hetero


In [173]:
# Change medium composition (acetate)

medium = model.medium
medium['EX_ac_e'] = 1000
model.medium = medium
model.medium

{'EX_h_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 1.0,
 'EX_so4_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_na1_e': 10.0,
 'EX_photonVis_e': 80.0,
 'EX_o2_e': 10.0,
 'EX_co2_e': 2.0,
 'EX_ac_e': 1000}

In [174]:
# Add heterologous reactions and metabolites

from cobra import Reaction, Metabolite

### Create metabolite ###
metabolite1 = Metabolite('jet_juice_c')
metabolite1.name = 'Jet Juice'
metabolite1.formula = 'C29H38O2'
metabolite1.compartment = 'c'
model.add_metabolites([metabolite1])

### Define 1st reaction ###
reaction1 = Reaction('jet_fuel')
reaction1.name = 'iPKS'
reaction1.lower_bound = 0.  # This makes the reaction reversible
reaction1.upper_bound = 1000.  # This is the default

# Add metabolites to the reaction
reaction1.add_metabolites({
    model.metabolites.get_by_id("ac_c"): -11.0,
    model.metabolites.get_by_id("nadp_c"): 11.0,
    model.metabolites.get_by_id("amet_c"): -7.0,
    model.metabolites.get_by_id("ahcys_c"): 7.0,
    model.metabolites.get_by_id("nadph_c"): -11.0,
    model.metabolites.get_by_id("h_c"): 11.0,
    model.metabolites.get_by_id("jet_juice_c"): 1.0
})

model.add_reactions([reaction1])

print(reaction1.reaction)

11.0 ac_c + 7.0 amet_c + 11.0 nadph_c --> 7.0 ahcys_c + 11.0 h_c + jet_juice_c + 11.0 nadp_c


In [167]:
## DEBUGGING ##

for reaction in model.metabolites.accoa_c.reactions:
    print(reaction.id, reaction, reaction.name, "\n")

ACCOAtm ACCOAtm: accoa_c + coa_m <=> accoa_m + coa_c Acetyl-CoA:CoA antiporter, mitochondrial 

ACCOAtx ACCOAtx: accoa_c + coa_x <=> accoa_x + coa_c Acetyl-CoA:CoA antiporter, glyoxysomal 

AAAT AAAT: accoa_c + glu_L_c --> acglu_c + coa_c + h_c amino-acid N-acetyltransferase 

ACALD ACALD: acald_c + coa_c + nad_c --> accoa_c + h_c + nadh_c Acetaldehyde dehydrogenase (acetylating) 

MCDC MCDC: malcoa_c --> accoa_c + co2_c malonyl-CoA decarboxylase 

BCACT BCACT: ac_c + btcoa_c <=> accoa_c + but_c butanoyl-CoA:acetate CoA transferase 

ACGAM6PSi ACGAM6PSi: accoa_c + gam6p_c --> acgam6p_c + coa_c + h_c acetyl-CoA:D-glucosamine-6-phosphate N-acetyltransferase 

ACDHi ACDHi: accoa_c + 2.0 h_c + 2.0 nadh_c --> coa_c + etoh_c + 2.0 nad_c acetaldehyde-CoA dehydrogenase 

CS CS: accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c citrate synthase 

MHGS MHGS: aacoa_c + accoa_c + h2o_c --> coa_c + hmgcoa_c hydroxymethylglutaryl-coa synthase 

ACACT ACACT: 2.0 accoa_c --> aacoa_c + coa_c Acetyl-CoA C

In [162]:
model.reactions.get_by_id('EX_ac_e').lower_bound = 0.
model.reactions.get_by_id('EX_ac_e').upper_bound = 0.

In [165]:
model.reactions.get_by_id('EX_ac_e')

Reaction identifier,EX_ac_e
Name,Acetate exchange
Memory address,0x7f86402226a0
Stoichiometry,ac_e --> Acetate -->
GPR,
Lower bound,0.0
Upper bound,0.0


In [175]:
# Add objective function

model.objective = 'jet_fuel'

In [176]:
# Simulate the model

solution = model.optimize()
solution

,fluxes,reduced_costs
AAT,0.0,0.0
ALATLm,0.0,0.0
ASNTLm,0.0,0.0
ASPO,0.0,0.0
ASPOm,0.0,0.0
...,...,...
PYDXS,0.0,0.0
Biomass_Chlamy_auto,0.0,0.0
Biomass_Chlamy_mixo,0.0,0.0
Biomass_Chlamy_hetero,0.0,0.0
